In [8]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import pandas as pd
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
from pathlib import Path
import csv
# Import the backtrader platform
import backtrader as bt

In [9]:
path = os.getcwd()
# print(path)
data_path = '../data/processed_ETH_OHLCV.csv'
temp = os.path.join(os.path.dirname('__file__'), data_path)
path = os.path.normpath(temp)

print(path)

../data/processed_ETH_OHLCV.csv


In [10]:
data = bt.feeds.GenericCSVData(
        dataname=path,
        nullvalue=0.0,
        # Do not pass values before this date
        fromdate=datetime.datetime(2018, 8, 8),
        # Do not pass values before this date
        todate=datetime.datetime(2020, 9, 9),
        dtformat=('%Y-%m-%d-%H:%M:%S'),
        datetime=0,
        time=-1,
        high=3,
        low=4,
        open=2,
        close=1,
        volume=-1,
        openinterest=-1
        # Do not pass values after this date
        # reverse=False
        )

In [11]:
class TestStrategy(bt.Strategy):
    lines = ('signal',)
    params = (('log_file', './backtest.csv'),
        ('p1', 4),
        ('p2', 8))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        # print('%s, %s' % (dt.isoformat(), txt))
        with Path(self.p.log_file).open('a') as f:
            log_writer = csv.writer(f)
            log_writer.writerow([dt.isoformat()] + txt.split(','))


    def __init__(self):
        # sma1 = btind.SimpleMovingAverage(self.data)
        # Keep a reference to the "close" line in the data[0] dataseries
        # sma1 = bt.indicators.SMA(period=self.p.p1)
        # sma2 = bt.indicators.SMA(period=self.p.p2)
        self.dataclose = self.datas[0].close
        # buy_sig = sma1 -sma2 > 0

    def next(self):
        # Simply log the closing price of the series from the reference
        # if self.data.dataclose > 350 :
        #     self.buy()
        # else:
        #      self.sell()
        self.log('Close, %.2f' % self.dataclose[0])

In [12]:
cerebro = bt.Cerebro()
    # Add a strategy
cerebro.addstrategy(TestStrategy)
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere

0

In [ ]:
cerebro.adddata(data)

    # Set our desired cash start
cerebro.broker.setcash(100000.0)

In [14]:
   # Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
cerebro.run()

    # Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
